In [1]:
import numpy as np
import math

In [65]:
class Value:
    
    def __init__(self, data, _children=()):
        self.data = data
        self._prev = set(_children)
        self._backward_fn = lambda: None
        self.grad = 0


    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other))

        def f():
            self.grad += out.grad
            other.grad += out.grad
        out._backward_fn = f

        return out
    

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        
        out = Value(self.data * other.data, (self, other))

        def f():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward_fn = f

        return out
    
    
    def __neg__(self): # -self
        return self * -1
    

    def __radd__(self, other): # other + self
        return self + other


    def __sub__(self, other): # self - other
        return self + (-other)


    def __rsub__(self, other): # other - self
        return other + (-self)


    def __rmul__(self, other): # other * self
        return self * other


    def __truediv__(self, other): # self / other
        return self * other**-1


    def __rtruediv__(self, other): # other / self
        return other * self**-1


    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,))

        def f():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward_fn = f

        return out
    
    
    def exp(self):
        out = Value(math.exp(self.data), (self, ))

        def f():
            self.grad += out.data * out.grad
        out._backward_fn = f

        return out
    
    def tanh(self):
        x = math.exp(2 * self.data)
        out = Value((x - 1) / (x + 1), (self, ))

        def f():
            self.grad += (1 - out.data ** 2) * out.grad
        out._backward_fn = f

        return out
    

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,))

        def f():
            self.grad += (out.data > 0) * out.grad
        out._backward_fn = f

        return out
    

    def backward(self):
        topologic, visited = list(), set()
        
        def build_topologic(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topologic(child)
                topologic.append(v)
        
        build_topologic(self)

        self.grad = 1
        for v in reversed(topologic):
            v._backward_fn()

In [66]:
import random 


class Neuron:

    def __init__(self, input_size: int):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(input_size)]
        self.b = Value(1)

    
    def __call__(self, x):
        out = sum((w_i * x_i for w_i, x_i in zip(self.w, x)), self.b)  # <w, x> + bias
        return out.tanh()
    

    def parameters(self):
        return self.w + [self.b]
    

class Layer:
    
    def __init__(self, input_size: int, output_size: int):
        self.neurons = [Neuron(input_size) for _ in range(output_size)]

    
    def __call__(self, x):
        out = [neuron(x) for neuron in self.neurons]
        return out if len(out) > 1 else out[0]
    

    def parameters(self):
        return [param for neuron in self.neurons for param in neuron.parameters()]


class MultiLayeredPerceptron:

    def __init__(self, input_size: int, output_size: list[int]):
        sizes = [input_size] + output_size
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(output_size))]

    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)

        return x
    

    def parameters(self):
        return [param for layer in self.layers for param in layer.parameters()]


In [69]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 

n = MultiLayeredPerceptron(input_size=3, output_size=[4, 4, 1])


In [ ]:
for k in range(20):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    for p in n.parameters():
        p.grad = 0
    
    loss.backward()

    for p in n.parameters():
        p.data += -0.05 * p.grad 


    print(k, loss.data)

0 6.002780218874915
1 4.721894024896821
2 3.2738538978251492
3 1.8897638018642917
4 2.5278914808201614
5 2.039584020603124
6 0.29245413912680807
7 0.5860552387613589
8 0.3167730721956112
9 0.10014234317284162
10 0.021532462264073116
11 0.0041082454718244085
12 0.002179587642527131
13 1.2328484930689734
14 5.449246867998356e-05
15 3.981583173747954
16 7.704279263831992
17 4.385786543430161
18 3.986683624488429
19 2.0936334259161886
